In [1]:
'''
差分内容

ベースライン作成
時系列クロスバリデーション

複数model保存・読み出し・予測→重み付き平均
重み付き平均はデータ量が多いほど重みを付けたがもっと適切なものを探る

PB score
0.56260
'''

'\n差分内容\n\nベースライン作成\n時系列クロスバリデーション\n\n複数model保存・読み出し・予測→重み付き平均\n重み付き平均はデータ量が多いほど重みを付けたがもっと適切なものを探る\n\n'

In [2]:
# main module
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling as pdp
import seaborn as sns

import lightgbm as lgb

from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error

import datetime
import gc
import logging
import pickle
from pickle import load
import sys, os

# -------------------------------------独自モジュール-------------------------------------
sys.path.append('../src/') #モジュールが入っているディレクトリのパスを指定
import eda
import maprepro as mpre
import maprepro2 as mpre2
# import config
# from utils import setup_logger, ModelFactory
# -------------------------------------独自モジュール-------------------------------------

In [3]:
path = '../../input'
sample = pd.read_csv(f'{path}/sample_submission.csv')
store = pd.read_csv(f'{path}/store.csv')
test = pd.read_csv(f'{path}/test.csv')
train = pd.read_csv(f'{path}/train.csv')

C:\Users\hayatotominaga\anaconda3\envs\rake4\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# periodの作成
# periodのセット
# - dateを残したままやって最後date削除


In [5]:
def mk_ymd(df):
    df['year'] = df.Date.apply(lambda x: x.split('-')[0]).astype(np.int16)
    df['month'] = df.Date.apply(lambda x: x.split('-')[1]).astype(np.int16)
    df['day'] = df.Date.apply(lambda x: x.split('-')[2]).astype(np.int8)
    df = df.sort_values('Date')
    return df
train = mk_ymd(train)
test = mk_ymd(test)

# 時系列データであり、時間に沿って変数periodを設定したとする
def mk_period(df,testdata=False):
    if testdata == False:
        df['period'] = np.arange(0, len(df)) // (len(df) // 4)
        df['period'] = np.clip(df['period'], 0, 3)
        return df
    else:
        df['period'] = 4
        return df


train = mk_period(train)
test = mk_period(test,testdata=True)

In [6]:
train['StateHoliday'] = train.StateHoliday.astype('category')
test['StateHoliday'] = test.StateHoliday.astype('category')

target = ['Sales']
notuse = ['Id','Date','Open']
use = ['Store','DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday','year','month','day','period']
categorical_features = ['StateHoliday']

train_y = train[target]
train_x = train[use]
test_x = test[use]

In [7]:
import warnings
warnings.simplefilter('ignore')
train_x = eda.reduce_mem_usage(train_x)

start size(BEFORE): 56.27 Mb
Mem. usage decreased to 19.40 Mb (AFTER:65.5% reduction)


In [8]:

# --------------------------------------------------
import maprepro2 as mpre2
NOW,TMP_DIR = mpre2.mk_dir()
gc.collect()
# --------------------------------------------------

va_period_list = [1, 2, 3]
for va_period in va_period_list:
    print('i ================================================================== ',va_period)
    is_tr = train_x['period'] < va_period
    is_va = train_x['period'] == va_period
    tr_x, va_x = train_x[is_tr], train_x[is_va]
    tr_y, va_y = train_y[is_tr], train_y[is_va]
    
    lgb_train = lgb.Dataset(tr_x, tr_y)
    lgb_eval = lgb.Dataset(va_x, va_y)

    # ハイパーパラメータの設定
    params = {'objective': 'regression',
                'seed': 71,
                'verbose': 1,
                'metrics': 'rmse',
                'force_col_wise':'true' # メモリが足りないから
                }
    num_round = 100

    # 学習の実行
    # カテゴリ変数をパラメータで指定している
    # バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
    categorical_features = categorical_features
    model = lgb.train(params, lgb_train, num_boost_round=num_round,
                    categorical_feature=categorical_features,
                    valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_eval],
                    )


    tmpfile = f'{TMP_DIR}/trained_model{va_period}.pkl'
    pickle.dump(model, open(tmpfile, 'wb'))

    # バリデーションデータでのスコアの確認
    va_pred = model.predict(va_x); va_pred = va_pred.reshape(-1, 1)

    # Open=0の日はすべてSalesが0で、0以外を予測することはありえないので0で更新(モデルは自然数を予測しているみたいだが、
    # Open=0ならSales=0をモデルに教えられていないということ)
    tmp =va_y['Sales']==0
    va_pred[tmp]=0
    va_pred=va_pred+1; va_y=va_y+1

    # --------------------------------------------------
    # score = mean_squared_error(va_y, va_pred)
    RMSPE = np.sqrt(np.mean(((  (va_y-va_pred)/va_y)**2) )).values
    RMSPE = RMSPE.astype(float)[0] # printのために単一の数値にしたかった
    # score = log_loss(va_y, va_pred)
    print('RMSPE RMSPE RMSPE RMSPE RMSPE RMSPE: {:.4f}'.format(RMSPE))
    # --------------------------------------------------

    del model # 学習済みモデルを削除


i ==================================================================  1
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 254302, number of used features: 8
[LightGBM] [Info] Start training from score 5576.839962
[1]	train's rmse: 3512.84	valid's rmse: 3644.08
[2]	train's rmse: 3335.43	valid's rmse: 3478
[3]	train's rmse: 3184.13	valid's rmse: 3335.7
[4]	train's rmse: 3055.07	valid's rmse: 3214.79
[5]	train's rmse: 2946.45	valid's rmse: 3114.04
[6]	train's rmse: 2855.12	valid's rmse: 3029.43
[7]	train's rmse: 2775.91	valid's rmse: 2954.98
[8]	train's rmse: 2710.06	valid's rmse: 2893.86
[9]	train's rmse: 2654.18	valid's rmse: 2840.49
[10]	train's rmse: 2608.38	valid's rmse: 2797.55
[11]	train's rmse: 2570.76	valid's rmse: 2760.34
[12]	train's rmse: 2536.69	valid's rmse: 2727.66
[13]	train's rmse: 2510.62	valid's rmse: 2701.75
[14]	train's rmse: 2489.38	valid's rmse: 2681.85
[15]	train's rmse: 2468.82	valid's rmse: 2661.58
[16]	train's rmse: 2451.

In [9]:
modeldict = mpre2.load_models(TMP_DIR)

load開始
load終わりました


In [10]:
pred_dict = mpre2.mk_pred_dict(modeldict,test_x)

予測リストが入った辞書作成開始
予測リストが入った辞書作成終了


In [11]:
pred_wei_average = mpre2.wei_average(pred_dict,len(test_x))

複数モデルを使用した重み付き平均予測を開始します
重み付き平均pred_wei_averageがreturnされます
重み付き平均pred_wei_averageがreturnされました


In [12]:
subpred = pd.DataFrame(pred_wei_average)
# subpred.sort_values('Sales')

sub = pd.concat([test['Id'],subpred],axis=1)
sub.columns = ['Id','Sales']

In [13]:
mpre2.mk_output(df=sub,NOW=NOW,PRACTICE=False)